# Face Verification — Main (Jetson Nano compatible)

This notebook provides a single runnable workflow that: 

- Captures an image from a local USB webcam (or falls back to Colab browser capture),
- Loads a pretrained embedding model (TensorRT engine on Jetson, or Keras on other machines),
- Builds a small gallery from reference images and runs a verification demo,
- Runs a final gender-detection step using DeepFace (optional).

Notes: On Jetson Nano prefer a TensorRT engine (`model.trt`). If you don't have one, the notebook will try to load `face_embedding_model_CLEAN.h5` (Keras). See `README_Jetson.md` for conversion steps.

In [ ]:
# Install runtime packages if needed (uncomment when running interactively)
# On Jetson, TensorRT & CUDA are provided by the system; avoid reinstalling them via pip.
# The following installs are optional and may be skipped if already present.
# !python3 -m pip install -r requirements.txt
# For gender detection:
# !pip install deepface --quiet

In [ ]:
# Standard imports
import os
import sys
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Project helpers (we added these to the repo)
from camera_utils import capture_image

# Try to import TensorRT helper; if not available we'll fall back to Keras
TRT_AVAILABLE = False
try:
    from trt_inference import TRTInfer
    import tensorrt as _trt  # noqa: F401
    TRT_AVAILABLE = True
except Exception:
    TRT_AVAILABLE = False

# Try to import tensorflow (only needed if falling back to Keras model)
TF_AVAILABLE = False
try:
    import tensorflow as tf
    TF_AVAILABLE = True
except Exception:
    TF_AVAILABLE = False

In [ ]:
# Detect platform heuristics (Jetson if tensorrt available or on aboard)
def is_jetson():
    # Simple heuristic: presence of TensorRT and CUDA on the system or the 'JETSON_SERIAL' env var
    if TRT_AVAILABLE:
        return True
    if os.path.exists('/etc/nv_tegra_release') or os.path.exists('/usr/bin/jetson_release'):
        return True
    return False

ON_JETSON = is_jetson()
print('TRT_AVAILABLE=', TRT_AVAILABLE, 'TF_AVAILABLE=', TF_AVAILABLE, 'ON_JETSON=', ON_JETSON)

In [ ]:
# MODEL LOADING: prefer TensorRT engine (model.trt) on Jetson, else load Keras .h5 extractor
ENGINE_PATH = 'model.trt'
KERAS_H5 = 'face_embedding_model_CLEAN.h5'
feature_extractor = None
trt_infer = None

if ON_JETSON and os.path.exists(ENGINE_PATH):
    print('Loading TensorRT engine from', ENGINE_PATH)
    try:
        trt_infer = TRTInfer(ENGINE_PATH, input_shape=(128,128,3))
        print('✅ Loaded TRT engine.')
    except Exception as e:
        print('❌ TRT engine load failed:', e)
        trt_infer = None

if trt_infer is None:
    # fallback to Keras model if available
    if os.path.exists(KERAS_H5) and TF_AVAILABLE:
        print('Loading Keras model from', KERAS_H5)
        try:
            feature_extractor = tf.keras.models.load_model(KERAS_H5, compile=False)
            print('✅ Loaded Keras feature extractor.')
        except Exception as e:
            print('❌ Failed to load Keras model:', e)
    else:
        print('No TRT engine or Keras model found. Please provide model.trt or face_embedding_model_CLEAN.h5')

In [ ]:
# Preprocessing and embedding helpers
def preprocess_for_model(img, target_size=(128,128)):
    # img: BGR (OpenCV) or RGB? We'll accept BGR and convert to RGB
    if img is None:
        return None
    if img.shape[2] == 3:
        # convert BGR->RGB
        img = img[:, :, ::-1]
    img = cv2.resize(img, (target_size[1], target_size[0]), interpolation=cv2.INTER_LINEAR)
    img = img.astype('float32') / 255.0
    return img

def get_embedding_from_image(img):
    # img: HxWxC BGR (from cv2.imread or capture), returns 1-D numpy vector
    pre = preprocess_for_model(img, target_size=(128,128))
    if pre is None:
        return None
    if trt_infer is not None:
        return trt_infer.predict(pre)
    if feature_extractor is not None:
        # Keras expects batch dimension
        inp = np.expand_dims(pre, axis=0)
        emb = feature_extractor.predict(inp, verbose=0)[0]
        return emb
    raise RuntimeError('No model available for inference')

In [ ]:
# Build a small gallery from reference folder (if available)
REF_DIR = './data_extracted/ref'
gallery = {}
if os.path.exists(REF_DIR):
    for root, _, files in os.walk(REF_DIR):
        for f in files:
            if not f.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            path = os.path.join(root, f)
            try:
                img = cv2.imread(path)
                emb = get_embedding_from_image(img)
                if emb is None:
                    continue
                pid = os.path.splitext(f)[0].split('__')[0]
                # store first reference per person (simple demo)
                if pid not in gallery:
                    gallery[pid] = { 'path': path, 'emb': emb }
            except Exception as e:
                print('Error processing', path, e)

print('Gallery size:', len(gallery))

In [ ]:
# Simple verification demo: capture a photo and compare to a random gallery entry
if not gallery:
    print('No gallery found; please populate', REF_DIR, 'and re-run this cell.')
else:
    # Select a reference entry
    ref_pid, ref_data = next(iter(gallery.items()))
    ref_path = ref_data['path']
    print('Using reference:', ref_pid, ref_path)
    # Capture a query image from local USB webcam
    query_path = 'captured_query.jpg'
    try:
        capture_image(query_path, camera_index=0, timeout=5, interactive=False)
        print('Captured', query_path)
    except Exception as e:
        print('Capture failed:', e)

    # Load and embed
    ref_img = cv2.imread(ref_path)
    query_img = cv2.imread(query_path)
    emb_ref = ref_data['emb']
    emb_query = get_embedding_from_image(query_img)
    dist = np.linalg.norm(emb_ref - emb_query)
    threshold = 0.6
    verdict = 'MATCH' if dist < threshold else 'REJECT'
    print(f'Distance={dist:.4f} -> {verdict}')

    # Show images side-by-side
    ref_rgb = cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB)
    query_rgb = cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(1,2,figsize=(10,5))
    ax[0].imshow(ref_rgb); ax[0].set_title(f'Ref: {ref_pid}'); ax[0].axis('off')
    ax[1].imshow(query_rgb); ax[1].set_title(f'Query (D={dist:.4f})'); ax[1].axis('off')
    plt.show()

In [ ]:
# =========================================================
# 🧠 GENDER DETECTION SECTION (FINAL FIXED VERSION)
# =========================================================
# Install deepface if needed (uncomment to run)
# !pip install deepface --quiet

from deepface import DeepFace
import tensorflow as tf
import os
import matplotlib.pyplot as plt

def detect_gender(image_path):
    try:
        # Clear previous TensorFlow models to avoid KerasTensor conflicts
        tf.keras.backend.clear_session()

        # Run gender analysis safely
        result = DeepFace.analyze(
            img_path=image_path,
            actions=['gender'],
            enforce_detection=False
        )
        # DeepFace.analyze returns a dict for a single image in newer versions, or a list in some versions
        if isinstance(result, list):
            r = result[0]
        else:
            r = result
        # `dominant_gender` is commonly returned; `gender` field may contain confidences
        gender = r.get('dominant_gender', 'Unknown')
        gender_conf_obj = r.get('gender', {})
        # Try to extract confidence if available
        confidence = 0.0
        if isinstance(gender_conf_obj, dict):
            confidence = gender_conf_obj.get(gender, 0.0) if gender in gender_conf_obj else list(gender_conf_obj.values())[0] if gender_conf_obj else 0.0
    except Exception as e:
        print(f'⚠️ Gender detection failed for {image_path}: {e}')
        gender = 'Unknown'
        confidence = 0.0
    return gender, confidence

# Use the reference and query paths from the previous verification step (if they exist)
try:
    ref_path  # noqa: F821
except NameError:
    # Fallback: pick any file from gallery if available
    if gallery:
        ref_path = next(iter(gallery.values()))['path']
    else:
        raise RuntimeError('No reference image available for gender detection')

try:
    query_path  # noqa: F821
except NameError:
    query_path = 'captured_query.jpg'

# Run detection
ref_gender, ref_conf = detect_gender(ref_path)
query_gender, query_conf = detect_gender(query_path)

# --- Display the images with detected genders ---
ref_img_display = cv2.cvtColor(cv2.imread(ref_path), cv2.COLOR_BGR2RGB)
query_img_display = cv2.cvtColor(cv2.imread(query_path), cv2.COLOR_BGR2RGB)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
fig.suptitle('Gender Detection Results', fontsize=16, fontweight='bold')

axes[0].imshow(ref_img_display)
axes[0].set_title(f'Reference\nGender: {ref_gender.capitalize()} ({ref_conf:.1f}%)', fontsize=12)
axes[0].axis('off')

axes[1].imshow(query_img_display)
axes[1].set_title(f'Query\nGender: {query_gender.capitalize()} ({query_conf:.1f}%)', fontsize=12)
axes[1].axis('off')

plt.tight_layout(rect=[0, 0.03, 1, 0.9])
plt.show()

print('
--- 🧩 GENDER DETECTION SUMMARY ---')
print(f'• Reference Image ({os.path.basename(ref_path)}): {ref_gender} ({ref_conf:.1f}%)')
print(f'• Query Image ({os.path.basename(query_path)}): {query_gender} ({query_conf:.1f}%)')

## Notes & Next Steps
- If you plan to run on Jetson Nano, build the TensorRT engine on the Nano following `README_Jetson.md`.
- If the notebook is run interactively, ensure the top cells that install packages are executed only once.
- If you want, I can patch the original `Face_Verification.ipynb` to import from this consolidated `main.ipynb` flow or replace the duplicate capture/embedding cells across the notebook.